Start with entering credentials

In [130]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import sklearn
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import json
import os
import time

cid = '1905d1c9ab8e4a58bffa9ae1aad1f623'
secret = '80519e5da1c1413dbefe71a5c15434a2'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# auth_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
# sp = spotipy.Spotify(auth_manager=auth_manager)

sp.artist('0TnOYISbd1XYRBk9myaseg')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
 'followers': {'href': None, 'total': 9647815},
 'genres': ['dance pop', 'miami hip hop', 'pop', 'pop rap'],
 'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
 'id': '0TnOYISbd1XYRBk9myaseg',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb2dc40ac263ef07c16a95af4e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051742dc40ac263ef07c16a95af4e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1782dc40ac263ef07c16a95af4e',
   'width': 160}],
 'name': 'Pitbull',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}

In [131]:
artist_name = []
track_name = []
track_id = []
artist_id = []
album_id = []
genre = []
for i in range(0,200,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        album_id.append(t['album']['id'])
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        genre.append(sp.artist(t['artists'][0]['id'])['genres'])

In [132]:
track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : artist_id ,'track_name' : track_name, 'track_id' : track_id, 'album_id' : album_id, 'genre' : genre})
print(track_dataframe.shape)
track_dataframe.head()

(200, 6)


,artist_name,artist_id,track_name,track_id,album_id,genre
0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]"
1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]"
2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]"
3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop]
4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]"


In [133]:
def get_audio_features(track_ids):
    tracks_len = len(track_ids)
    chunk_size = int(tracks_len/tracks_len)
    audio_features = []
    begin = 0
    for track_id in track_ids:
        audio_features.append(sp.audio_features(track_ids[begin:begin+chunk_size]))
        begin += chunk_size
    return audio_features

audio_features = get_audio_features(track_dataframe['track_id'])

In [134]:
def merge_feature_dicts(audio_features):
    headers = audio_features[0][0].keys()
    list_of_dicts = []

    # merge all the dictionaries into one dictionary
    for dictionaries in audio_features:
        for d in dictionaries:
            list_of_dicts.append(d)


    # convert dictionary into dataframe
    d = {}
    for k in headers:
        d[k] = tuple(d[k] for d in list_of_dicts)

    feature_df = pd.DataFrame(d)
    return feature_df

feature_df = merge_feature_dicts(audio_features)


def concat_features_and_df(track_dataframe, feature_df):
    # concatinate dataframes with eachother
    track_dataframe = pd.concat([track_dataframe, feature_df],axis='columns')
    track_dataframe = track_dataframe.drop(['type', 'id'], axis=1)
    return track_dataframe

track_dataframe = concat_features_and_df(track_dataframe, feature_df)
track_dataframe

,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,duration_ms,time_signature
0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]",0.614,0.680,6,-4.580,...,0.3690,0.000002,0.1150,0.70700,149.959,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4
1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]",0.323,0.737,10,-4.008,...,0.1420,0.000162,0.1630,0.15500,113.260,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4
2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]",0.897,0.662,1,-6.903,...,0.0852,0.000000,0.5340,0.38900,112.511,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4
3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop],0.515,0.479,3,-7.458,...,0.5440,0.005980,0.1910,0.28400,88.964,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4
4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,-9.309,...,0.0355,0.000000,0.1080,0.56200,119.957,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Manchester Orchestra,5wFXmYsg3KFJ8BDsQudJ4f,The Gold - Phoebe Bridgers Version,7qcXUzPwoxSBFxjTbNrV0B,5pCfjopxazLdvmLae6My9w,[indie rock],0.299,0.399,10,-12.176,...,0.8160,0.001170,0.1470,0.30200,158.253,spotify:track:7qcXUzPwoxSBFxjTbNrV0B,https://api.spotify.com/v1/tracks/7qcXUzPwoxSB...,https://api.spotify.com/v1/audio-analysis/7qcX...,233227,3
196,SadBoyProlific,0inkJpjAKDnQOv2VHBCU0Z,Dead and Cold,33LizV83s7z1gwtmLiKlmv,48ydOppyPEiQuWz8MbGLuU,"[sad lo-fi, sad rap]",0.622,0.467,2,-9.889,...,0.8720,0.000002,0.0931,0.71600,87.967,spotify:track:33LizV83s7z1gwtmLiKlmv,https://api.spotify.com/v1/tracks/33LizV83s7z1...,https://api.spotify.com/v1/audio-analysis/33Li...,89625,5
197,"Tyler, The Creator",4V8LLVI7PbaPR0K2TGSxFF,Potato Salad,1jzIJcHCXneHw7ojC6LXiF,02KFVEA5tnY0YnypmWY1tz,"[hip hop, rap]",0.538,0.612,2,-6.020,...,0.3080,0.000000,0.2130,0.68400,168.197,spotify:track:1jzIJcHCXneHw7ojC6LXiF,https://api.spotify.com/v1/tracks/1jzIJcHCXneH...,https://api.spotify.com/v1/audio-analysis/1jzI...,182493,4
198,Shoreline Mafia,4tYSBptyGeVyZsk8JC4JHZ,"Bands (feat. OhGeesy, Fenix Flexin & Master Kato)",1n8wr8tRHs5jmBxNWXedcn,2rSKi2QJ6v22f464Uq3ose,"[cali rap, trap, underground hip hop, viral rap]",0.900,0.385,1,-12.189,...,0.3710,0.000000,0.1420,0.48800,99.964,spotify:track:1n8wr8tRHs5jmBxNWXedcn,https://api.spotify.com/v1/tracks/1n8wr8tRHs5j...,https://api.spotify.com/v1/audio-analysis/1n8w...,174336,4


In [135]:
features_df = track_dataframe.iloc[:,6:17]

cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10))])
cluster_pipeline.fit(features_df)
features_df['cluster'] = cluster_pipeline.predict(features_df)

In [136]:
tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])
genre_embedding = tsne_pipeline.fit_transform(features_df)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = track_dataframe['genre']
projection['cluster'] = features_df['cluster']
projection

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 200 samples in 0.001s...
[t-SNE] Computed neighbors for 200 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 1.606755
[t-SNE] Computed conditional probabilities in 0.006s
[t-SNE] Iteration 50: error = 61.6302528, gradient norm = 0.5080013 (50 iterations in 0.043s)
[t-SNE] Iteration 100: error = 61.0857697, gradient norm = 0.5164979 (50 iterations in 0.034s)
[t-SNE] Iteration 150: error = 61.2986221, gradient norm = 0.5047422 (50 iterations in 0.036s)
[t-SNE] Iteration 200: error = 61.0556183, gradient norm = 0.5450816 (50 iterations in 0.035s)
[t-SNE] Iteration 250: error = 62.5189285, gradient norm = 0.4893187 (50 iterations in 0.032s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 62.518929
[t-SNE] Iteration 300: error = 0.9415598, gradient norm = 0.0056791 (50 iterations in 0.042s)
[t-SNE] Iteration 350: error = 0.7094111, gradient norm = 0.0044938 (

,x,y,genres,cluster
0,-9.962620,-6.889171,"[contemporary country, country]",1
1,-4.975088,-8.785481,"[canadian contemporary r&b, canadian pop, pop]",1
2,10.166483,-11.149918,"[atl hip hop, atl trap, rap, trap]",9
3,6.566764,0.094547,[viral pop],7
4,-8.890446,7.127192,"[atl hip hop, atl trap, rap, trap]",6
...,...,...,...,...
195,5.226490,5.141581,[indie rock],0
196,-9.606667,8.709510,"[sad lo-fi, sad rap]",6
197,12.266706,-5.475076,"[hip hop, rap]",8
198,-8.809504,7.911796,"[cali rap, trap, underground hip hop, viral rap]",6


In [137]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'


fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

In [138]:
data_folder = 'data/spotify_million_playlist_dataset/data'
##################################################################################################
##################################################################################################
artist_name = []
track_name = []
track_id = []
artist_id = []
album_id = []
genre = []


for filename in os.listdir(data_folder):
    f = os.path.join(data_folder, filename)
    break

with open(f) as json_file:
    data = json.load(json_file)
    
for playlist in data['playlists'][0:15]:    
    tracks = playlist['tracks']
    for track in tracks:
        artist_name.append(track['artist_name'])
        track_name.append(track['track_name'])
        track_id.append(track['track_uri'].split(':')[2])
        art_id = track['artist_uri'].split(':')[2]
        artist_id.append(art_id)
        album_id.append(track['album_uri'].split(':')[2])
        genre.append(sp.artist(art_id)['genres'])
        
big_dataframe = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : artist_id ,'track_name' : track_name, 'track_id' : track_id, 'album_id' : album_id, 'genre' : genre})
print(big_dataframe.shape)
big_dataframe.head()

(1043, 6)


,artist_name,artist_id,track_name,track_id,album_id,genre
0,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Boots of Spanish Leather,6QHYEZlm9wyfXfEM1vSu1P,7DZeLXvr9eTVpyI1OlqtcS,"[classic rock, country rock, folk, folk rock, ..."
1,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Mr. Tambourine Man,3RkQ3UwOyPqpIiIvGVewuU,1lPoRKSgZHQAYXxzBsOQ7v,"[classic rock, country rock, folk, folk rock, ..."
2,Loggins & Messina,7emRV8AluG3d4e5T0DZiK9,Danny's Song,0ju1jP0cSPJ8tmojYBEI89,5BWgJaesMjpJWCTU9sgUPf,"[album rock, classic rock, country rock, folk,..."
3,Bob Dylan,74ASZWbe4lXaubB36ztrGX,A Hard Rain's A-Gonna Fall,7ny2ATvjtKszCpLpfsGnVQ,0o1uFxZ1VTviqvNaYkTJek,"[classic rock, country rock, folk, folk rock, ..."
4,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Blowin' In the Wind,18GiV1BaXzPVYpp9rmOg0E,0o1uFxZ1VTviqvNaYkTJek,"[classic rock, country rock, folk, folk rock, ..."


In [139]:
audio_features = get_audio_features(big_dataframe['track_id'])
feature_df = merge_feature_dicts(audio_features)
new_dataframe = concat_features_and_df(big_dataframe, feature_df)
new_dataframe

,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,duration_ms,time_signature
0,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Boots of Spanish Leather,6QHYEZlm9wyfXfEM1vSu1P,7DZeLXvr9eTVpyI1OlqtcS,"[classic rock, country rock, folk, folk rock, ...",0.615,0.1140,8,-17.285,...,0.92600,0.000004,0.1660,0.401,101.189,spotify:track:6QHYEZlm9wyfXfEM1vSu1P,https://api.spotify.com/v1/tracks/6QHYEZlm9wyf...,https://api.spotify.com/v1/audio-analysis/6QHY...,277107,4
1,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Mr. Tambourine Man,3RkQ3UwOyPqpIiIvGVewuU,1lPoRKSgZHQAYXxzBsOQ7v,"[classic rock, country rock, folk, folk rock, ...",0.390,0.3100,5,-14.545,...,0.79400,0.000000,0.3790,0.584,177.515,spotify:track:3RkQ3UwOyPqpIiIvGVewuU,https://api.spotify.com/v1/tracks/3RkQ3UwOyPqp...,https://api.spotify.com/v1/audio-analysis/3RkQ...,330533,4
2,Loggins & Messina,7emRV8AluG3d4e5T0DZiK9,Danny's Song,0ju1jP0cSPJ8tmojYBEI89,5BWgJaesMjpJWCTU9sgUPf,"[album rock, classic rock, country rock, folk,...",0.505,0.2580,11,-12.193,...,0.80500,0.000000,0.1200,0.641,141.003,spotify:track:0ju1jP0cSPJ8tmojYBEI89,https://api.spotify.com/v1/tracks/0ju1jP0cSPJ8...,https://api.spotify.com/v1/audio-analysis/0ju1...,254305,4
3,Bob Dylan,74ASZWbe4lXaubB36ztrGX,A Hard Rain's A-Gonna Fall,7ny2ATvjtKszCpLpfsGnVQ,0o1uFxZ1VTviqvNaYkTJek,"[classic rock, country rock, folk, folk rock, ...",0.567,0.1980,4,-18.681,...,0.91900,0.000871,0.1350,0.321,92.467,spotify:track:7ny2ATvjtKszCpLpfsGnVQ,https://api.spotify.com/v1/tracks/7ny2ATvjtKsz...,https://api.spotify.com/v1/audio-analysis/7ny2...,412200,4
4,Bob Dylan,74ASZWbe4lXaubB36ztrGX,Blowin' In the Wind,18GiV1BaXzPVYpp9rmOg0E,0o1uFxZ1VTviqvNaYkTJek,"[classic rock, country rock, folk, folk rock, ...",0.380,0.0993,2,-20.567,...,0.91400,0.000023,0.0605,0.440,174.874,spotify:track:18GiV1BaXzPVYpp9rmOg0E,https://api.spotify.com/v1/tracks/18GiV1BaXzPV...,https://api.spotify.com/v1/audio-analysis/18Gi...,165427,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,Jenny Lewis,5cMVRrisBpDkXCVG48epED,Barking at the Moon,575TC82P638HnynAEbAXuU,4pRN2ctEESw8NdxpWie66B,[indie pop],0.565,0.5420,2,-9.569,...,0.37100,0.000000,0.1390,0.528,111.064,spotify:track:575TC82P638HnynAEbAXuU,https://api.spotify.com/v1/tracks/575TC82P638H...,https://api.spotify.com/v1/audio-analysis/575T...,197293,4
1039,Owl City,07QEuhtrNmmZ0zEcqE9SF6,When Can I See You Again?,5ZkAx8zjLiSs1nMmBwJoZS,0aqJGQ73RTLu3yLgcdU9JC,"[indietronica, pop, pop rock]",0.638,0.8310,11,-6.812,...,0.02630,0.000000,0.0491,0.388,127.975,spotify:track:5ZkAx8zjLiSs1nMmBwJoZS,https://api.spotify.com/v1/tracks/5ZkAx8zjLiSs...,https://api.spotify.com/v1/audio-analysis/5ZkA...,218093,4
1040,Bryan Adams,3Z02hBLubJxuFJfhacLSDc,Here I Am - Spirit: Stallion Of The Cimarron/S...,6oOJU64nYMU4ljUiMCGeHe,1vSVsVvRpSLB6HzqT7raEd,"[canadian pop, canadian singer-songwriter, cla...",0.581,0.7720,8,-7.170,...,0.00852,0.000030,0.0837,0.172,98.962,spotify:track:6oOJU64nYMU4ljUiMCGeHe,https://api.spotify.com/v1/tracks/6oOJU64nYMU4...,https://api.spotify.com/v1/audio-analysis/6oOJ...,284893,4
1041,Michael Giacchino,4kLvhMAuCloLxoP1aVM7Lr,"The Glory Days - From ""The Incredibles"" / Score",5MzfX0lViNEiGcNoCJtjiq,72qvk5vtwrICeV739WEGwX,"[orchestral soundtrack, soundtrack]",0.334,0.2620,5,-14.193,...,0.73200,0.388000,0.2350,0.145,80.446,spotify:track:5MzfX0lViNEiGcNoCJtjiq,https://api.spotify.com/v1/tracks/5MzfX0lViNEi...,https://api.spotify.com/v1/audio-analysis/5Mzf...,212133,4


In [140]:
new_dataframe = pd.concat([track_dataframe, new_dataframe])
new_dataframe = new_dataframe.reset_index()
new_dataframe

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]",0.614,0.680,6,...,0.36900,0.000002,0.1150,0.707,149.959,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4
1,1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]",0.323,0.737,10,...,0.14200,0.000162,0.1630,0.155,113.260,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4
2,2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]",0.897,0.662,1,...,0.08520,0.000000,0.5340,0.389,112.511,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4
3,3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop],0.515,0.479,3,...,0.54400,0.005980,0.1910,0.284,88.964,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4
4,4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,...,0.03550,0.000000,0.1080,0.562,119.957,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,1038,Jenny Lewis,5cMVRrisBpDkXCVG48epED,Barking at the Moon,575TC82P638HnynAEbAXuU,4pRN2ctEESw8NdxpWie66B,[indie pop],0.565,0.542,2,...,0.37100,0.000000,0.1390,0.528,111.064,spotify:track:575TC82P638HnynAEbAXuU,https://api.spotify.com/v1/tracks/575TC82P638H...,https://api.spotify.com/v1/audio-analysis/575T...,197293,4
1239,1039,Owl City,07QEuhtrNmmZ0zEcqE9SF6,When Can I See You Again?,5ZkAx8zjLiSs1nMmBwJoZS,0aqJGQ73RTLu3yLgcdU9JC,"[indietronica, pop, pop rock]",0.638,0.831,11,...,0.02630,0.000000,0.0491,0.388,127.975,spotify:track:5ZkAx8zjLiSs1nMmBwJoZS,https://api.spotify.com/v1/tracks/5ZkAx8zjLiSs...,https://api.spotify.com/v1/audio-analysis/5ZkA...,218093,4
1240,1040,Bryan Adams,3Z02hBLubJxuFJfhacLSDc,Here I Am - Spirit: Stallion Of The Cimarron/S...,6oOJU64nYMU4ljUiMCGeHe,1vSVsVvRpSLB6HzqT7raEd,"[canadian pop, canadian singer-songwriter, cla...",0.581,0.772,8,...,0.00852,0.000030,0.0837,0.172,98.962,spotify:track:6oOJU64nYMU4ljUiMCGeHe,https://api.spotify.com/v1/tracks/6oOJU64nYMU4...,https://api.spotify.com/v1/audio-analysis/6oOJ...,284893,4
1241,1041,Michael Giacchino,4kLvhMAuCloLxoP1aVM7Lr,"The Glory Days - From ""The Incredibles"" / Score",5MzfX0lViNEiGcNoCJtjiq,72qvk5vtwrICeV739WEGwX,"[orchestral soundtrack, soundtrack]",0.334,0.262,5,...,0.73200,0.388000,0.2350,0.145,80.446,spotify:track:5MzfX0lViNEiGcNoCJtjiq,https://api.spotify.com/v1/tracks/5MzfX0lViNEi...,https://api.spotify.com/v1/audio-analysis/5Mzf...,212133,4


In [141]:
features_df = new_dataframe.iloc[:,7:17]
features_df


cluster_pipeline.fit(features_df)
features_df['cluster'] = cluster_pipeline.predict(features_df)
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,cluster
0,0.614,0.680,6,-4.580,1,0.0289,0.36900,0.000002,0.1150,0.707,5
1,0.323,0.737,10,-4.008,1,0.0338,0.14200,0.000162,0.1630,0.155,0
2,0.897,0.662,1,-6.903,0,0.2920,0.08520,0.000000,0.5340,0.389,7
3,0.515,0.479,3,-7.458,1,0.0261,0.54400,0.005980,0.1910,0.284,4
4,0.963,0.346,5,-9.309,0,0.5300,0.03550,0.000000,0.1080,0.562,9
...,...,...,...,...,...,...,...,...,...,...,...
1238,0.565,0.542,2,-9.569,1,0.0328,0.37100,0.000000,0.1390,0.528,4
1239,0.638,0.831,11,-6.812,0,0.0810,0.02630,0.000000,0.0491,0.388,8
1240,0.581,0.772,8,-7.170,1,0.0295,0.00852,0.000030,0.0837,0.172,0
1241,0.334,0.262,5,-14.193,0,0.0391,0.73200,0.388000,0.2350,0.145,2


In [142]:
# tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])
genre_embedding = tsne_pipeline.fit_transform(features_df)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = new_dataframe['genre']
projection['cluster'] = features_df['cluster']
projection

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1243 samples in 0.001s...
[t-SNE] Computed neighbors for 1243 samples in 0.064s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1243
[t-SNE] Computed conditional probabilities for sample 1243 / 1243
[t-SNE] Mean sigma: 0.990979
[t-SNE] Computed conditional probabilities in 0.033s
[t-SNE] Iteration 50: error = 69.1098480, gradient norm = 0.1919974 (50 iterations in 0.181s)
[t-SNE] Iteration 100: error = 64.1017151, gradient norm = 0.1191721 (50 iterations in 0.122s)
[t-SNE] Iteration 150: error = 63.4811172, gradient norm = 0.1120976 (50 iterations in 0.117s)
[t-SNE] Iteration 200: error = 63.3930283, gradient norm = 0.1051082 (50 iterations in 0.111s)
[t-SNE] Iteration 250: error = 63.3712120, gradient norm = 0.0893700 (50 iterations in 0.111s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.371212
[t-SNE] Iteration 300: error = 0.9969869, gradient norm = 0.0007466 (50 iterations in 0.122s)

,x,y,genres,cluster
0,2.034969,10.507359,"[contemporary country, country]",5
1,-68.066116,14.856197,"[canadian contemporary r&b, canadian pop, pop]",0
2,36.758381,-1.546835,"[atl hip hop, atl trap, rap, trap]",7
3,-13.290733,-1.586432,[viral pop],4
4,48.247948,1.478830,"[atl hip hop, atl trap, rap, trap]",9
...,...,...,...,...
1238,-17.121319,-1.701675,[indie pop],4
1239,42.276695,-21.086308,"[indietronica, pop, pop rock]",8
1240,-64.182793,14.535100,"[canadian pop, canadian singer-songwriter, cla...",0
1241,-7.394207,-22.219971,"[orchestral soundtrack, soundtrack]",2


In [143]:
fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

In [191]:
# Now let's check how we can find a recommendation for your own playlist
my_playlist = sp.playlist('7nL2047lhk158BzXuAAx4Z?si=8a293b5c06b44b64')

In [192]:
artist_name = []
track_name = []
track_id = []
artist_id = []
album_id = []
genre = []

my_tracks = my_playlist['tracks']['items']
my_tracks
for track in my_tracks:
    # print(track['track']['artists'][0]['name'])
    artist_name.append(track['track']['artists'][0]['name'])
    track_name.append(track['track']['name'])
    track_id.append(track['track']['id'])
    art_id = track['track']['artists'][0]['id']
    artist_id.append(art_id)
    album_id.append(track['track']['album']['id'])
    genre.append(sp.artist(art_id)['genres'])
    
my_track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : artist_id ,'track_name' : track_name, 'track_id' : track_id, 'album_id' : album_id, 'genre' : genre})
print(my_track_dataframe.shape)
my_track_dataframe.head()

(100, 6)


,artist_name,artist_id,track_name,track_id,album_id,genre
0,Sunni Colón,4tRZRfhgfLEIUI9mw9Njy2,Baby I Don't Mind,40YkgUfMvXowQd8WuwCqJG,1HRzxvSMJSc6sNHrU2JA1W,"[alternative r&b, indie r&b, indie soul]"
1,Channel Tres,4cUkGQyhLFqKHBtL58HYVp,Weedman,1sQeLo71tyRhY7TqHRiSQ9,43kOtF27RadDQ5xn3EAJ3d,[la pop]
2,,0LyfQWJT6nXafLPZqxe9Of,,5MvCurNMpM3WCCSs40cd37,6GK7MOPqacKl84T0r6Rctc,[]
3,Blood Orange,6LEeAFiJF8OuPx747e1wxR,Gold Teeth,3BbWvLOFaxx0npzxfRixyO,3KLLRiCnqYvt78H52TeZ1k,"[alternative r&b, art pop, chillwave, indie so..."
4,Free Nationals,4596e2d3KmYzAeVenjCxfj,Shibuya (feat. Syd),6WVSnyKQGzs1fosa2I3FMQ,53oqFs4q8sfqH6IPiyleEN,"[alternative r&b, indie soul]"


In [193]:
my_audio_features = get_audio_features(my_track_dataframe['track_id'])
my_feature_df = merge_feature_dicts(my_audio_features)
my_new_track_dataframe = concat_features_and_df(my_track_dataframe, my_feature_df)

In [196]:
# my_track_dataframe = my_track_dataframe.reset_index()
my_new_track_dataframe = pd.concat([new_dataframe, my_new_track_dataframe])
my_new_dataframe = my_new_track_dataframe.reset_index()
my_new_track_dataframe.reindex()
my_new_track_dataframe['index'] = range(0, len(my_new_track_dataframe))
# my_new_track_dataframe.drop(column)

In [197]:
my_new_track_dataframe.set_index(my_new_track_dataframe.iloc[:,0])
my_new_track_dataframe.reset_index(drop=True, inplace=True)
my_new_track_dataframe.iloc[:,0]


0          0
1          1
2          2
3          3
4          4
        ... 
1338    1338
1339    1339
1340    1340
1341    1341
1342    1342
Name: index, Length: 1343, dtype: int64

In [198]:
my_features_df = my_new_track_dataframe.iloc[:,7:17]

cluster_pipeline.fit(my_features_df)
my_features_df['cluster'] = cluster_pipeline.predict(my_features_df)

In [199]:
# tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])
genre_embedding = tsne_pipeline.fit_transform(my_features_df)
my_projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
my_projection['genres'] = my_new_track_dataframe['genre']
my_projection['cluster'] = my_features_df['cluster']

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1343 samples in 0.001s...
[t-SNE] Computed neighbors for 1343 samples in 0.067s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1343
[t-SNE] Computed conditional probabilities for sample 1343 / 1343
[t-SNE] Mean sigma: 0.969535
[t-SNE] Computed conditional probabilities in 0.032s
[t-SNE] Iteration 50: error = 72.3509903, gradient norm = 0.1553907 (50 iterations in 0.196s)
[t-SNE] Iteration 100: error = 65.6896133, gradient norm = 0.0788439 (50 iterations in 0.123s)
[t-SNE] Iteration 150: error = 65.4596558, gradient norm = 0.0793570 (50 iterations in 0.119s)
[t-SNE] Iteration 200: error = 65.4064941, gradient norm = 0.0586266 (50 iterations in 0.133s)
[t-SNE] Iteration 250: error = 65.3990402, gradient norm = 0.0604228 (50 iterations in 0.147s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.399040
[t-SNE] Iteration 300: error = 1.1057125, gradient norm = 0.0008498 (50 iterations in 0.127s)

In [200]:
fig = px.scatter(my_projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

In [201]:
x_mean = np.mean(projection['x'])
y_mean = np.mean(projection['y'])
print('the mean of x:', x_mean)
print('the mean of y:', y_mean)

the mean of x: -2.5710287
the mean of y: -1.2734226


In [202]:
# projection['genres'].unique()
encoder = sklearn.preprocessing.LabelEncoder()
encoder.fit(my_projection['genres'].astype(str))
genres_encoding = encoder.fit_transform(my_projection['genres'].astype(str))
# projection['genres'].loc[0]

In [203]:
genres_encoding

array([297, 243, 146, ...,   1, 545, 202])

In [204]:
my_new_track_dataframe

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]",0.614,0.680,6,...,0.36900,0.000002,0.1150,0.707,149.959,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4
1,1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]",0.323,0.737,10,...,0.14200,0.000162,0.1630,0.155,113.260,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4
2,2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]",0.897,0.662,1,...,0.08520,0.000000,0.5340,0.389,112.511,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4
3,3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop],0.515,0.479,3,...,0.54400,0.005980,0.1910,0.284,88.964,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4
4,4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,...,0.03550,0.000000,0.1080,0.562,119.957,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1338,Isaiah Rashad,6aaMZ3fcfLv4tEbmY7bjRM,Why Worry,0M5pEUmRD3b4nP31L88IdL,7GAOMkrIBSwqvC28f8bXTj,"[hip hop, rap, tennessee hip hop, underground ...",0.287,0.870,1,...,0.00291,0.000000,0.0609,0.681,81.162,spotify:track:0M5pEUmRD3b4nP31L88IdL,https://api.spotify.com/v1/tracks/0M5pEUmRD3b4...,https://api.spotify.com/v1/audio-analysis/0M5p...,150562,4
1339,1339,Madvillain,2aoFQUeHD1U7pL098lRsDU,Accordion,5jAvbp8kEnRPToi4Bzcvzt,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.578,0.631,9,...,0.02430,0.000000,0.0665,0.716,96.332,spotify:track:5jAvbp8kEnRPToi4Bzcvzt,https://api.spotify.com/v1/tracks/5jAvbp8kEnRP...,https://api.spotify.com/v1/audio-analysis/5jAv...,118853,4
1340,1340,Madvillain,2aoFQUeHD1U7pL098lRsDU,All Caps,1Y2ExJJ9Dmb9po8K0ybSj3,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.358,0.889,1,...,0.26600,0.000015,0.1720,0.666,85.610,spotify:track:1Y2ExJJ9Dmb9po8K0ybSj3,https://api.spotify.com/v1/tracks/1Y2ExJJ9Dmb9...,https://api.spotify.com/v1/audio-analysis/1Y2E...,130480,4
1341,1341,slowthai,3r1XkJ7vCs8kHBSzGvPLdP,feel away (feat. James Blake & Mount Kimbie),0pauM2SXIq7pAQlmpYq3N6,5fAID5buWAhtqIdpVDwvar,"[rap, uk alternative hip hop, uk hip hop, unde...",0.404,0.532,0,...,0.68300,0.158000,0.1640,0.339,159.864,spotify:track:0pauM2SXIq7pAQlmpYq3N6,https://api.spotify.com/v1/tracks/0pauM2SXIq7p...,https://api.spotify.com/v1/audio-analysis/0pau...,200933,4


In [367]:
projection_df = pd.concat([my_new_track_dataframe, my_projection],axis='columns')

In [368]:
projection_df['genre_encoding'] = genres_encoding.tolist()
projection_df

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,uri,track_href,analysis_url,duration_ms,time_signature,x,y,genres,cluster,genre_encoding
0,0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]",0.614,0.680,6,...,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4,-22.955866,30.710554,"[contemporary country, country]",9,297
1,1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]",0.323,0.737,10,...,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4,-4.739160,-36.433163,"[canadian contemporary r&b, canadian pop, pop]",1,243
2,2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]",0.897,0.662,1,...,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4,1.958061,18.167496,"[atl hip hop, atl trap, rap, trap]",6,146
3,3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop],0.515,0.479,3,...,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4,2.141904,-17.136715,[viral pop],4,562
4,4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,...,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4,7.682563,23.391647,"[atl hip hop, atl trap, rap, trap]",8,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1338,Isaiah Rashad,6aaMZ3fcfLv4tEbmY7bjRM,Why Worry,0M5pEUmRD3b4nP31L88IdL,7GAOMkrIBSwqvC28f8bXTj,"[hip hop, rap, tennessee hip hop, underground ...",0.287,0.870,1,...,spotify:track:0M5pEUmRD3b4nP31L88IdL,https://api.spotify.com/v1/tracks/0M5pEUmRD3b4...,https://api.spotify.com/v1/audio-analysis/0M5p...,150562,4,27.325556,17.821012,"[hip hop, rap, tennessee hip hop, underground ...",2,435
1339,1339,Madvillain,2aoFQUeHD1U7pL098lRsDU,Accordion,5jAvbp8kEnRPToi4Bzcvzt,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.578,0.631,9,...,spotify:track:5jAvbp8kEnRPToi4Bzcvzt,https://api.spotify.com/v1/tracks/5jAvbp8kEnRP...,https://api.spotify.com/v1/audio-analysis/5jAv...,118853,4,54.048588,17.816713,"[abstract hip hop, alternative hip hop, hip ho...",0,1
1340,1340,Madvillain,2aoFQUeHD1U7pL098lRsDU,All Caps,1Y2ExJJ9Dmb9po8K0ybSj3,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.358,0.889,1,...,spotify:track:1Y2ExJJ9Dmb9po8K0ybSj3,https://api.spotify.com/v1/tracks/1Y2ExJJ9Dmb9...,https://api.spotify.com/v1/audio-analysis/1Y2E...,130480,4,-35.423965,-13.066487,"[abstract hip hop, alternative hip hop, hip ho...",2,1
1341,1341,slowthai,3r1XkJ7vCs8kHBSzGvPLdP,feel away (feat. James Blake & Mount Kimbie),0pauM2SXIq7pAQlmpYq3N6,5fAID5buWAhtqIdpVDwvar,"[rap, uk alternative hip hop, uk hip hop, unde...",0.404,0.532,0,...,spotify:track:0pauM2SXIq7pAQlmpYq3N6,https://api.spotify.com/v1/tracks/0pauM2SXIq7p...,https://api.spotify.com/v1/audio-analysis/0pau...,200933,4,2.995662,-15.078292,"[rap, uk alternative hip hop, uk hip hop, unde...",4,545


In [207]:
projection_df.loc[projection_df['track_name'] == 'All Caps']

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,uri,track_href,analysis_url,duration_ms,time_signature,x,y,genres,cluster,genre_encoding
1340,1340,Madvillain,2aoFQUeHD1U7pL098lRsDU,All Caps,1Y2ExJJ9Dmb9po8K0ybSj3,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.358,0.889,1,...,spotify:track:1Y2ExJJ9Dmb9po8K0ybSj3,https://api.spotify.com/v1/tracks/1Y2ExJJ9Dmb9...,https://api.spotify.com/v1/audio-analysis/1Y2E...,130480,4,-35.423965,-13.066487,"[abstract hip hop, alternative hip hop, hip ho...",2,1


In [221]:
projection_df.loc[(projection_df['cluster'] == 8)]

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,uri,track_href,analysis_url,duration_ms,time_signature,x,y,genres,cluster,genre_encoding
4,4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,...,spotify:track:6vN77lE9LK6HP2DewaN6HZ,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4,7.682563,23.391647,"[atl hip hop, atl trap, rap, trap]",8,146
9,9,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Pure Cocaine,577YBGuskWkVDCxZrLRB4v,4iF44RCCybJtkBkMoqrLW2,"[atl hip hop, atl trap, rap, trap]",0.964,0.487,10,...,spotify:track:577YBGuskWkVDCxZrLRB4v,https://api.spotify.com/v1/tracks/577YBGuskWkV...,https://api.spotify.com/v1/audio-analysis/577Y...,154024,4,7.206479,25.187954,"[atl hip hop, atl trap, rap, trap]",8,146
21,21,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,Violent Crimes,3s7MCdXyWmwjdcWh7GWXas,2Ek1q2haOnxVqhvVKqMvJe,"[chicago rap, rap]",0.669,0.419,1,...,spotify:track:3s7MCdXyWmwjdcWh7GWXas,https://api.spotify.com/v1/tracks/3s7MCdXyWmwj...,https://api.spotify.com/v1/audio-analysis/3s7M...,215320,4,8.747743,26.055624,"[chicago rap, rap]",8,264
31,31,Metro Boomin,0iEtIxbK0KxaSlF7G42ZOp,10 Freaky Girls (with 21 Savage),0AluA5RNsa4Cx6XRhf2hWZ,3IO8IPjwXuzPJnoaqkwYrj,"[hip hop, rap, trap]",0.810,0.353,6,...,spotify:track:0AluA5RNsa4Cx6XRhf2hWZ,https://api.spotify.com/v1/tracks/0AluA5RNsa4C...,https://api.spotify.com/v1/audio-analysis/0Alu...,208293,4,10.655449,24.744770,"[hip hop, rap, trap]",8,436
63,63,Ski Mask The Slump God,2rhFzFmezpnW82MNqEKVry,Faucet Failure,1ThmUihH9dF8EV08ku5AXN,0z0z4DcXhHiobX5ZKAw8Qn,"[miami hip hop, rap]",0.935,0.552,10,...,spotify:track:1ThmUihH9dF8EV08ku5AXN,https://api.spotify.com/v1/tracks/1ThmUihH9dF8...,https://api.spotify.com/v1/audio-analysis/1Thm...,145627,4,5.950048,24.491228,"[miami hip hop, rap]",8,501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,1314,Blood Orange,6LEeAFiJF8OuPx747e1wxR,Smoke - Remix,22Fvc0gkf7ZCNQEQ2Oxj6D,2gHdg26eOQzbHM9DN15BDW,"[alternative r&b, art pop, chillwave, indie so...",0.702,0.493,6,...,spotify:track:22Fvc0gkf7ZCNQEQ2Oxj6D,https://api.spotify.com/v1/tracks/22Fvc0gkf7ZC...,https://api.spotify.com/v1/audio-analysis/22Fv...,280473,4,6.790784,29.528763,"[alternative r&b, art pop, chillwave, indie so...",8,83
1316,1316,Daniel Caesar,20wkVLutqVOYrc0kxFs7rA,CYANIDE REMIX (feat. Koffee),5HGU08aDQkOjwogqE3SwhG,4kYeULYvGSPb5igBdYlJ5S,"[canadian contemporary r&b, pop, r&b]",0.476,0.592,10,...,spotify:track:5HGU08aDQkOjwogqE3SwhG,https://api.spotify.com/v1/tracks/5HGU08aDQkOj...,https://api.spotify.com/v1/audio-analysis/5HGU...,221132,4,-7.826383,32.310604,"[canadian contemporary r&b, pop, r&b]",8,245
1321,1321,Kill Bill: The Rapper,1t9KewtUvFMq6q7FRX1nHC,Black Coffee,1AAMj0EJJdOkErtr4HtEhS,2FQKpBA2aLzkgvIV7WwazZ,"[lo-fi rap, meme rap]",0.374,0.720,5,...,spotify:track:1AAMj0EJJdOkErtr4HtEhS,https://api.spotify.com/v1/tracks/1AAMj0EJJdOk...,https://api.spotify.com/v1/audio-analysis/1AAM...,280554,4,8.602779,30.337700,"[lo-fi rap, meme rap]",8,490
1324,1324,Maeta,2EwyKG76iX4Pp5HhAD6SKO,Teen Scene feat. Buddy,7rNjupJkjsc3ANGeW0RCYj,3bLZlNsCcoc4Ih0FtqPvkF,"[alternative r&b, chill r&b, indie r&b]",0.336,0.741,11,...,spotify:track:7rNjupJkjsc3ANGeW0RCYj,https://api.spotify.com/v1/tracks/7rNjupJkjsc3...,https://api.spotify.com/v1/audio-analysis/7rNj...,250362,4,-9.290773,28.706099,"[alternative r&b, chill r&b, indie r&b]",8,89


In [369]:
my_track_dataframe.to_csv('dataframes/track_dataframe_without_features', encoding='utf-8', index=False)
my_new_track_dataframe.to_csv('dataframes/track_dataframe_complete', encoding='utf-8', index=False)

In [370]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
genres_list = projection_df['genres'].to_list()
len(genres_list)

1343

In [371]:
model = Word2Vec(genres_list, min_count=1)
model.wv.similarity('atl hip hop', 'trap')
model.wv['atl hip hop']
test_wv = model.wv['atl hip hop']
word_vectors = []

for genre in genres_list:
    if genre == []:
        word_vectors.append(np.zeros_like(test_wv).mean())
    else:
        word_vectors.append(model.wv[genre].mean())

In [372]:
# projection_df = projection_df.drop('genre_encoding', axis=1)
projection_df['genre_embedding'] = word_vectors

In [374]:
projection_df

,index,artist_name,artist_id,track_name,track_id,album_id,genre,danceability,energy,key,...,track_href,analysis_url,duration_ms,time_signature,x,y,genres,cluster,genre_encoding,genre_embedding
0,0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,1IR2nlwX6YVTXXeu2qzoWO,"[contemporary country, country]",0.614,0.680,6,...,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4,-22.955866,30.710554,"[contemporary country, country]",9,297,0.000560
1,1,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,4qZBW3f2Q8y0k1A84d4iAO,"[canadian contemporary r&b, canadian pop, pop]",0.323,0.737,10,...,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067,4,-4.739160,-36.433163,"[canadian contemporary r&b, canadian pop, pop]",1,243,0.000812
2,2,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,2yXnY2NiaZk9QiJJittS81,"[atl hip hop, atl trap, rap, trap]",0.897,0.662,1,...,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4,1.958061,18.167496,"[atl hip hop, atl trap, rap, trap]",6,146,0.001100
3,3,Joji,3MZsBdqDrRTJihTHQrO6Dq,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,34GQP3dILpyCN018y2k61L,[viral pop],0.515,0.479,3,...,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4,2.141904,-17.136715,[viral pop],4,562,0.000083
4,4,Lil Baby,5f7VJjfbwm532GiveGC0ZK,Yes Indeed,6vN77lE9LK6HP2DewaN6HZ,7yf9ZJyHRbp8sHtpDKfPpt,"[atl hip hop, atl trap, rap, trap]",0.963,0.346,5,...,https://api.spotify.com/v1/tracks/6vN77lE9LK6H...,https://api.spotify.com/v1/audio-analysis/6vN7...,142273,4,7.682563,23.391647,"[atl hip hop, atl trap, rap, trap]",8,146,0.001100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1338,Isaiah Rashad,6aaMZ3fcfLv4tEbmY7bjRM,Why Worry,0M5pEUmRD3b4nP31L88IdL,7GAOMkrIBSwqvC28f8bXTj,"[hip hop, rap, tennessee hip hop, underground ...",0.287,0.870,1,...,https://api.spotify.com/v1/tracks/0M5pEUmRD3b4...,https://api.spotify.com/v1/audio-analysis/0M5p...,150562,4,27.325556,17.821012,"[hip hop, rap, tennessee hip hop, underground ...",2,435,0.000847
1339,1339,Madvillain,2aoFQUeHD1U7pL098lRsDU,Accordion,5jAvbp8kEnRPToi4Bzcvzt,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.578,0.631,9,...,https://api.spotify.com/v1/tracks/5jAvbp8kEnRP...,https://api.spotify.com/v1/audio-analysis/5jAv...,118853,4,54.048588,17.816713,"[abstract hip hop, alternative hip hop, hip ho...",0,1,0.000538
1340,1340,Madvillain,2aoFQUeHD1U7pL098lRsDU,All Caps,1Y2ExJJ9Dmb9po8K0ybSj3,01FCoGEQ3NFWF4fHJzdiax,"[abstract hip hop, alternative hip hop, hip ho...",0.358,0.889,1,...,https://api.spotify.com/v1/tracks/1Y2ExJJ9Dmb9...,https://api.spotify.com/v1/audio-analysis/1Y2E...,130480,4,-35.423965,-13.066487,"[abstract hip hop, alternative hip hop, hip ho...",2,1,0.000538
1341,1341,slowthai,3r1XkJ7vCs8kHBSzGvPLdP,feel away (feat. James Blake & Mount Kimbie),0pauM2SXIq7pAQlmpYq3N6,5fAID5buWAhtqIdpVDwvar,"[rap, uk alternative hip hop, uk hip hop, unde...",0.404,0.532,0,...,https://api.spotify.com/v1/tracks/0pauM2SXIq7p...,https://api.spotify.com/v1/audio-analysis/0pau...,200933,4,2.995662,-15.078292,"[rap, uk alternative hip hop, uk hip hop, unde...",4,545,0.000551


In [375]:
projection_features = projection_df.iloc[:,7:18]

projection_features['embeddings'] = projection_df['genre_embedding']

cluster_pipeline.fit(projection_features)
projection_features['cluster'] = cluster_pipeline.predict(projection_features)
projection_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,embeddings,cluster
0,0.614,0.680,6,-4.580,1,0.0289,0.36900,0.000002,0.1150,0.707,149.959,0.000560,4
1,0.323,0.737,10,-4.008,1,0.0338,0.14200,0.000162,0.1630,0.155,113.260,0.000812,2
2,0.897,0.662,1,-6.903,0,0.2920,0.08520,0.000000,0.5340,0.389,112.511,0.001100,1
3,0.515,0.479,3,-7.458,1,0.0261,0.54400,0.005980,0.1910,0.284,88.964,0.000083,9
4,0.963,0.346,5,-9.309,0,0.5300,0.03550,0.000000,0.1080,0.562,119.957,0.001100,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,0.287,0.870,1,-6.689,0,0.2930,0.00291,0.000000,0.0609,0.681,81.162,0.000847,1
1339,0.578,0.631,9,-6.518,0,0.3090,0.02430,0.000000,0.0665,0.716,96.332,0.000538,1
1340,0.358,0.889,1,-8.045,1,0.3350,0.26600,0.000015,0.1720,0.666,85.610,0.000538,2
1341,0.404,0.532,0,-8.989,1,0.0409,0.68300,0.158000,0.1640,0.339,159.864,0.000551,9


In [376]:
genre_embedding = tsne_pipeline.fit_transform(projection_features)
projection_features = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection_features['genre'] = projection_df['genre']
projection_features['cluster'] = projection_df['cluster']

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/sayf/opt/anaconda3/envs/deepl/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1343 samples in 0.001s...
[t-SNE] Computed neighbors for 1343 samples in 0.072s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1343
[t-SNE] Computed conditional probabilities for sample 1343 / 1343
[t-SNE] Mean sigma: 1.135026
[t-SNE] Computed conditional probabilities in 0.028s
[t-SNE] Iteration 50: error = 72.0066986, gradient norm = 0.1700990 (50 iterations in 0.158s)
[t-SNE] Iteration 100: error = 68.4284210, gradient norm = 0.0997779 (50 iterations in 0.126s)
[t-SNE] Iteration 150: error = 68.1824112, gradient norm = 0.0802653 (50 iterations in 0.139s)
[t-SNE] Iteration 200: error = 68.1691284, gradient norm = 0.0964345 (50 iterations in 0.122s)
[t-SNE] Iteration 250: error = 68.1409836, gradient norm = 0.1012333 (50 iterations in 0.133s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 68.140984
[t-SNE] Iteration 300: error = 1.3178689, gradient norm = 0.0008991 (50 iterations in 0.132s)

In [378]:
fig = px.scatter(projection_features, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genre'])
fig.show()